Дан граф текста, множество вершин совпадает со словарем текста, связи являются взвешенными и показывают как часто два слова находились в одном окне длины N. Найти слова с наибольшей мерой посредничества.



In [75]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk import everygrams
nltk.download('punkt')
import re
import collections
import itertools
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 8.2MB 3.9MB/s 


In [40]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Читаем какой-нибудь текст, например, Джека Лондона

In [41]:
def read_file (filename):
    with open (filename, encoding = 'utf-8') as file:
        text = file.read()
    return text 

In [42]:
filename = 'D_London.txt'

In [43]:
text = read_file (filename)

Возьмем для примера окна длины 4

In [44]:
words = []
n = 4

Берем из него только первую главу, чтобы побыстрее работало

In [45]:
def part_of_text (text):
    chapter_1 = []
    chapters = re.split(r'I+\.', text)
    chapter_1 = chapters[1]
    return chapter_1

In [46]:
chapter_1 = part_of_text (text)

Преобразуем его для удобства

In [47]:
chapter_1 = chapter_1.lower()

In [48]:
chapter_2 = re.sub('[^\w \n]+', '', chapter_1)
chapter = re.sub('\s+', ' ', chapter_2)

Токенизируем текст

In [49]:
tokenized_text = nltk.word_tokenize(chapter)

Превратим список токенов в список лемм и создадим n-граммы

In [50]:
lemmas_list = []

In [51]:
for word in tokenized_text:
  ana = morph.parse(word)[0].normal_form
  lemmas_list.append(ana)

In [52]:
def n_grams_list (tokenized_text, n):
    n_grams = list(ngrams(tokenized_text, n))
    return n_grams

Список кортежей окон длины n

In [53]:
n_grams = n_grams_list (lemmas_list, n)

Необходимые для дальнейших действий пустые списки

In [54]:
list_of_pairs = []
pairs_list = []
word_pairs = []

Делам список всевозможных комбинаций двух слов по каждому из окон 

In [55]:
for n_gram in n_grams:
  list_of_pairs.extend(list(itertools.combinations(n_gram, 2))) 

In [56]:
list_of_pairs

[('к', 'первобытный'),
 ('к', 'жизнь'),
 ('к', 'древние'),
 ('первобытный', 'жизнь'),
 ('первобытный', 'древние'),
 ('жизнь', 'древние'),
 ('первобытный', 'жизнь'),
 ('первобытный', 'древние'),
 ('первобытный', 'бродячий'),
 ('жизнь', 'древние'),
 ('жизнь', 'бродячий'),
 ('древние', 'бродячий'),
 ('жизнь', 'древние'),
 ('жизнь', 'бродячий'),
 ('жизнь', 'инстинкт'),
 ('древние', 'бродячий'),
 ('древние', 'инстинкт'),
 ('бродячий', 'инстинкт'),
 ('древние', 'бродячий'),
 ('древние', 'инстинкт'),
 ('древние', 'перетирать'),
 ('бродячий', 'инстинкт'),
 ('бродячий', 'перетирать'),
 ('инстинкт', 'перетирать'),
 ('бродячий', 'инстинкт'),
 ('бродячий', 'перетирать'),
 ('бродячий', 'цепь'),
 ('инстинкт', 'перетирать'),
 ('инстинкт', 'цепь'),
 ('перетирать', 'цепь'),
 ('инстинкт', 'перетирать'),
 ('инстинкт', 'цепь'),
 ('инстинкт', 'привычка'),
 ('перетирать', 'цепь'),
 ('перетирать', 'привычка'),
 ('цепь', 'привычка'),
 ('перетирать', 'цепь'),
 ('перетирать', 'привычка'),
 ('перетирать', 'и'),


Проблема в том, что в тех комбинациях есть варианты, например ('на', 'я') и ('я', 'на'), но по сути это одно и то же ребро, так как наш граф неориентированный. Превратим кортежи в списки и отсортируем каждый из них, тогда "названия" одного и того же ребра будет одинаково.

In [57]:
for element in list_of_pairs:
  pairs_list.append(list(element))

In [58]:
number1 = 0
for pair in pairs_list:
  number1 += 1
  pair.sort()

Это число отражает сумму количеств вхождений каждого из ребер

In [59]:
number1

19758

Далее мы переделаем опять списки в кортежи, так как списки не могут быть ключом словаря

In [60]:
for pair in pairs_list:
  word_pairs.append(tuple(pair))

Создадим частотный словарь

In [61]:
counter = collections.Counter(word_pairs)

In [62]:
counter

Counter({('к', 'первобытный'): 1,
         ('жизнь', 'к'): 1,
         ('древние', 'к'): 1,
         ('жизнь', 'первобытный'): 2,
         ('древние', 'первобытный'): 2,
         ('древние', 'жизнь'): 3,
         ('бродячий', 'первобытный'): 1,
         ('бродячий', 'жизнь'): 2,
         ('бродячий', 'древние'): 3,
         ('жизнь', 'инстинкт'): 1,
         ('древние', 'инстинкт'): 2,
         ('бродячий', 'инстинкт'): 3,
         ('древние', 'перетирать'): 1,
         ('бродячий', 'перетирать'): 2,
         ('инстинкт', 'перетирать'): 3,
         ('бродячий', 'цепь'): 1,
         ('инстинкт', 'цепь'): 2,
         ('перетирать', 'цепь'): 3,
         ('инстинкт', 'привычка'): 1,
         ('перетирать', 'привычка'): 2,
         ('привычка', 'цепь'): 3,
         ('и', 'перетирать'): 1,
         ('и', 'цепь'): 2,
         ('и', 'привычка'): 4,
         ('век', 'цепь'): 1,
         ('век', 'привычка'): 2,
         ('век', 'и'): 6,
         ('и', 'и'): 15,
         ('и', 'просыпаться'): 4,


Посмотрим, какое количество ребер получилось в нашем графе

In [63]:
num = 0

In [64]:
for key in counter:
  num += 1

In [65]:
num

8003

Далее приведем наш словарь к иному виду: сделаем список списков, в каждом из которых будет три элемента (две смежные вершины и число, которое отражает как часто эти два слова находились в одном окне длины n)

In [66]:
data = []

In [67]:
for pair, number in counter.items():
  edge = []
  for word in pair:
    edge.append(word)
  freq_dict = {}
  freq_dict['freq'] = number
  edge.append(freq_dict)
  data.append(edge)


In [68]:
data

[['к', 'первобытный', {'freq': 1}],
 ['жизнь', 'к', {'freq': 1}],
 ['древние', 'к', {'freq': 1}],
 ['жизнь', 'первобытный', {'freq': 2}],
 ['древние', 'первобытный', {'freq': 2}],
 ['древние', 'жизнь', {'freq': 3}],
 ['бродячий', 'первобытный', {'freq': 1}],
 ['бродячий', 'жизнь', {'freq': 2}],
 ['бродячий', 'древние', {'freq': 3}],
 ['жизнь', 'инстинкт', {'freq': 1}],
 ['древние', 'инстинкт', {'freq': 2}],
 ['бродячий', 'инстинкт', {'freq': 3}],
 ['древние', 'перетирать', {'freq': 1}],
 ['бродячий', 'перетирать', {'freq': 2}],
 ['инстинкт', 'перетирать', {'freq': 3}],
 ['бродячий', 'цепь', {'freq': 1}],
 ['инстинкт', 'цепь', {'freq': 2}],
 ['перетирать', 'цепь', {'freq': 3}],
 ['инстинкт', 'привычка', {'freq': 1}],
 ['перетирать', 'привычка', {'freq': 2}],
 ['привычка', 'цепь', {'freq': 3}],
 ['и', 'перетирать', {'freq': 1}],
 ['и', 'цепь', {'freq': 2}],
 ['и', 'привычка', {'freq': 4}],
 ['век', 'цепь', {'freq': 1}],
 ['век', 'привычка', {'freq': 2}],
 ['век', 'и', {'freq': 6}],
 ['и'

Что-то про степень посредничества и про сам граф 

In [69]:
G = nx.Graph()

In [70]:
G.add_edges_from(data)

In [49]:
G.edges(data=True)

EdgeDataView([('к', 'первобытный', {'freq': 1}), ('к', 'жизнь', {'freq': 1}), ('к', 'древний', {'freq': 1}), ('к', 'с', {'freq': 1}), ('к', 'весь', {'freq': 4}), ('к', 'сторона', {'freq': 3}), ('к', 'дом', {'freq': 3}), ('к', 'вели', {'freq': 2}), ('к', 'посыпать', {'freq': 1}), ('к', 'фунт', {'freq': 1}), ('к', 'вес', {'freq': 2}), ('к', 'если', {'freq': 3}), ('к', 'они', {'freq': 3}), ('к', 'ещё', {'freq': 3}), ('к', 'прибавить', {'freq': 1}), ('к', 'большой', {'freq': 1}), ('к', 'порка', {'freq': 2}), ('к', 'страсть', {'freq': 3}), ('к', 'китайский', {'freq': 5}), ('к', 'лотерея', {'freq': 5}), ('к', 'к', {'freq': 1}), ('к', 'тот', {'freq': 3}), ('к', 'же', {'freq': 2}), ('к', 'у', {'freq': 1}), ('к', 'быть', {'freq': 2}), ('к', 'равносильно', {'freq': 2}), ('к', 'приказание', {'freq': 3}), ('к', 'он', {'freq': 8}), ('к', 'удивление', {'freq': 2}), ('к', 'верёвка', {'freq': 1}), ('к', 'когда', {'freq': 1}), ('к', 'кидаться', {'freq': 3}), ('к', 'решётка', {'freq': 6}), ('к', 'дрожат

In [71]:
dict_BC = nx.betweenness_centrality(G)

Выведем топ-10 слов с наибольшей степенью посредничества 

In [72]:
for key in sorted(dict_BC, key=dict_BC.get, reverse = True)[:10]:
  print(key, dict_BC[key])

и 0.2691421422046305
он 0.2136053866548678
в 0.12095968021397838
бэк 0.07038377695534379
на 0.06670005572221421
с 0.05149479948739939
не 0.04549161872357748
быть 0.04532212315812836
они 0.039490940328677775
а 0.03580668739683459
